In [1]:
from bs4 import BeautifulSoup,Comment
import numpy as np
import pandas as pd
import regex as re
import requests
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import json
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import datetime

In [3]:
request_headers = {
    'accept':
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/601.3.9'
}

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

with requests.Session() as session:
    url = 'https://gasprices.aaa.com/'
    response = session.get(url, headers=request_headers)

browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=5)

False

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')
df = pd.DataFrame()

In [5]:
html_data = soup.find(class_="table-mob")
html_data2 = soup.find(class_="average-price")

date = [span.get_text() for span in html_data2.find_all("span")]
data = [td.get_text() for td in html_data.find_all("td")]

In [6]:
CurrentDate = []
GasPrice = []
DieselPrice = []
day = pd.to_datetime(date[0][11:])

CurrentDate.append(day)
GasPrice.append(round(float(data[1][1:]),2))
DieselPrice.append(round(float(data[4][1:]),2))

df['Date'] = CurrentDate
df['Regular'] = GasPrice
df['Diesel'] = DieselPrice

In [7]:
gas = pd.read_csv("./gas-diesel-prices.csv")
# gas = gas.drop(columns=['Unnamed: 3', 'Unnamed: 4'])
gas['Date'] = pd.to_datetime(gas['Date'])

In [8]:
gas = pd.concat([gas, df], ignore_index = True)

In [9]:
gas

,Unnamed: 0,Date,Regular,Diesel
0,0.0,2022-02-23,3.54,3.96
1,1.0,2022-02-24,3.54,3.96
2,2.0,2022-02-25,3.57,3.98
3,3.0,2022-02-26,3.60,4.00
4,4.0,2022-02-27,3.60,4.00
...,...,...,...,...
281,281.0,2022-12-04,3.41,5.10
282,282.0,2022-12-05,3.40,5.08
283,283.0,2022-12-06,3.38,5.06
284,284.0,2022-12-07,3.35,5.03


In [221]:
# gas = gas.drop(columns=['Unnamed: 0'])
gas

,Date,Regular,Diesel
0,2022-02-23,3.54,3.96
1,2022-02-24,3.54,3.96
2,2022-02-25,3.57,3.98
3,2022-02-26,3.60,4.00
4,2022-02-27,3.60,4.00
...,...,...,...
280,2022-12-03,3.43,5.11
281,2022-12-04,3.41,5.10
282,2022-12-05,3.40,5.08
283,2022-12-06,3.38,5.06


In [222]:
gas.to_csv('gas-diesel-prices.csv')